In [59]:
%reload_ext autoreload
%autoreload 2
import csv
from CIOGame import *
import pandas as pd
import numpy as np
from sklearn import linear_model, preprocessing, ensemble
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from gurobipy import *

# Reports in previous semesters

In [ ]:
file = 'CMO_Controlling-Tool.xlsx'
# Load spreadsheet
xl = pd.ExcelFile(file)
df=xl.parse('Marketing KPIs').drop(list(range(10,18)))
df=df.set_index([list(df.iloc[:,0])]).iloc[:,1:].transpose().drop(['Prozessqualität'],axis=1)
df

In [ ]:
cio_game.correlation_matrix(df)

In [ ]:
df_corr=df.corr()
df_corr

In [ ]:
df_corr.abs().sort_values(by=['     Global'],ascending=False)#.drop(["Marktanteil (New Savings)","Kundenzufriedenheit"])

# CIO regression
## Read data sets from .xlsm

In [267]:
cio_game=CIOGame(5)
df_cmo=cio_game.read_from_exel().T
df_cmo

Created CIOGame Object for period 5


,Car Financing Loans,Customer Savings,Requests to be processed LowPriceCars,Requests to be processed MidPriceCars,Requests to be processed HighPriceCars,Contracts succeeded LowPriceCars,Contracts succeeded MidPriceCars,Contracts succeeded HighPriceCars,Service Transactions to be processed loans,Service Transactions succeeded loans,...,Contracts succeeded savings,Service Transactions succeeded savings,Customer Satisfaction,Marketing Efficiency for product Car Financing Loans,Marketing Efficiency for product Savings Account,Marketing Expenditures Global,Marketing Expenditures Product Loans,Marketing Expenditures Product Savings,Interest Rate Car Financing Loans,Interest Rate Customer Savings
0,1.806205e+09,1.084270e+09,59391.8,74239.8,14848.0,18460.7,23075.9,4615.2,216749.4,173290.4,...,136564.3,156517.6,0.8332,0.7295,0.6915,4500000.0,2220000.0,1200000.0,0.120,0.030
1,2.595910e+09,9.625200e+08,56685.5,70856.9,14171.4,39239.7,49049.6,9809.9,311509.6,298465.7,...,172480.4,135516.1,0.8492,0.6938,0.6576,4500000.0,2020000.0,1400000.0,0.120,0.035
2,3.387190e+09,1.138230e+09,62877.9,78597.4,15719.5,41754.9,52193.6,10438.7,406463.3,327376.3,...,308382.6,162806.6,0.8681,0.7884,0.7680,4500000.0,2220000.0,1200000.0,0.123,0.035
3,3.732180e+09,2.021880e+09,46496.4,58120.5,11624.1,35495.5,44369.4,8873.9,447862.1,425422.3,...,772732.2,285108.1,0.8929,0.9085,0.9026,3300000.0,1100000.0,900000.0,0.130,0.032
4,4.475400e+09,3.038280e+09,54842.7,68553.3,13710.7,50091.1,62613.9,12522.8,537047.4,524852.8,...,1048853.7,428042.9,0.9269,0.9689,0.9684,3600000.0,1200000.0,1440000.0,0.137,0.030
5,5.574000e+09,5.433580e+09,63389.0,79236.3,15847.3,60304.8,75381.0,15076.2,668880.0,651918.1,...,2071513.3,668233.4,0.9663,0.9854,0.9854,4500000.0,1500000.0,5000000.0,0.147,0.023


In [268]:
df_cmo.keys()

Index(['Car Financing Loans', 'Customer Savings',
       'Requests to be processed LowPriceCars',
       'Requests to be processed MidPriceCars',
       'Requests to be processed HighPriceCars',
       'Contracts succeeded LowPriceCars', 'Contracts succeeded MidPriceCars',
       'Contracts succeeded HighPriceCars',
       'Service Transactions to be processed loans',
       'Service Transactions succeeded loans',
       'Requests to be processed savings', 'Contracts succeeded savings',
       'Service Transactions succeeded savings', 'Customer Satisfaction',
       'Marketing Efficiency for product Car Financing Loans',
       'Marketing Efficiency for product Savings Account',
       'Marketing Expenditures Global', 'Marketing Expenditures Product Loans',
       'Marketing Expenditures Product Savings',
       'Interest Rate Car Financing Loans', 'Interest Rate Customer Savings'],
      dtype='object', name='')

In [269]:
sum_contracts_succeeded_loans=df_cmo[[
    'Contracts succeeded LowPriceCars',
    'Contracts succeeded MidPriceCars',
    'Contracts succeeded HighPriceCars'
]].sum(axis=1).rename("Contracts succeeded Total")

df_loans=df_cmo[[
    "Car Financing Loans",
    'Service Transactions succeeded loans',
]]
df_loans=df_loans.T.append(sum_contracts_succeeded_loans).T
df_loans

,Car Financing Loans,Service Transactions succeeded loans,Contracts succeeded Total
0,1.806205e+09,173290.4,46151.8
1,2.595910e+09,298465.7,98099.2
2,3.387190e+09,327376.3,104387.2
3,3.732180e+09,425422.3,88738.8
4,4.475400e+09,524852.8,125227.8
5,5.574000e+09,651918.1,150762.0


## Find the best regression model

In [276]:
y=df_loans["Car Financing Loans"]
X=df_loans.drop(["Car Financing Loans"],axis=1)
regr_loans=cio_game.find_best_regr(X[:-1],y[:-1],X.iloc[len(X)-1],y.iloc[len(y)-1])

[0.0072008838816408627, 0.1981230339508222, 0.16317272810997352, 0.020450840665972025, 0.032614101431490683]
5574000000.0
5509205729.29
                               features  estimatedCoefficients
0  Service Transactions succeeded loans            6848.753335
1             Contracts succeeded Total            4187.009921


## Find the relationship of amount contracts in last period and amount of service transactions in this period

In [271]:
df_ta_loans=df_loans.drop(["Car Financing Loans"],axis=1).T.append(df_cmo["Service Transactions to be processed loans"].shift(-1).rename("Service Transactions to be processed loans next Period"))
df_ta_loans=df_ta_loans.append(df_cmo["Service Transactions succeeded loans"].shift(1).rename("Service Transactions to be processed loans last Period")).T
df_ta_loans

,Service Transactions succeeded loans,Contracts succeeded Total,Service Transactions to be processed loans next Period,Service Transactions to be processed loans last Period
0,173290.4,46151.8,311509.6,NaN
1,298465.7,98099.2,406463.3,173290.4
2,327376.3,104387.2,447862.1,298465.7
3,425422.3,88738.8,537047.4,327376.3
4,524852.8,125227.8,668880.0,425422.3
5,651918.1,150762.0,NaN,524852.8


In [272]:
y=df_ta_loans["Service Transactions to be processed loans next Period"][1:]
X=df_ta_loans.drop(["Service Transactions to be processed loans next Period"],axis=1)[1:]
regr_loans=cio_game.find_best_regr(X[:-1],y[:-1],X.iloc[len(X)-1],y.iloc[len(y)-1])

[0.1159725350443257, 0.059820206270812859, 0.028533388938561721, 0.058510362172537578]
nan
813679.61353
                                            features  estimatedCoefficients
0               Service Transactions succeeded loans               1.056644
1                          Contracts succeeded Total               0.093373
2  Service Transactions to be processed loans las...               0.081993


In [273]:
pd.DataFrame([df_cmo["Service Transactions to be processed loans"].shift(-1).rename("Service Transactions succeeded loans next Period"),df_cmo[[
    'Contracts succeeded LowPriceCars',
    'Contracts succeeded MidPriceCars',
    'Contracts succeeded HighPriceCars',
    'Service Transactions succeeded loans',
]].sum(axis=1).rename("Contracts succeeded Total")]).T

,Service Transactions succeeded loans next Period,Contracts succeeded Total
0,311509.6,219442.2
1,406463.3,396564.9
2,447862.1,431763.5
3,537047.4,514161.1
4,668880.0,650080.6
5,NaN,802680.1


# test

In [288]:
global_expenditure=np.array([4500000,4500000,4500000,3300000,3600000,4500000])

savings_intrests_rate=np.array([0.0300,0.0350,0.0350,0.0320,0.03,0.023])
savings_expenditure=np.array([1200000,1400000,1200000,900000,1440000,5000000])
savings_marketing_efficiency=np.array([0.6915,0.6576,0.7680,0.9026,0.9684,0.9864])

loans_intrests_rate=np.array([0.12,0.12,0.123,0.13,0.137,0.147])
loans_expenditure=np.array([2220000,2020000,2220000,1100000,1200000,1500000])
loans_marketing_efficiency=np.array([0.7295,0.6938,0.7884,0.9085,0.9689,0.9854])

savings_requests=np.array([902795.10,951090.40,993601.10,837795.40,1114117.8,2376077.1 ])
loans_requests=np.array([148479.60,141713.80 ,157194.80 ,116241.00,137106.70,158472.6])
#loans_requests=np.array([74239.80 ,70856.90 ,78597.40 ,58120.50 ,68553.30 ,79236.3])

df_savings=pd.DataFrame({
    "global_expenditure":global_expenditure,
    "savings_expenditure":savings_expenditure,
    "savings_intrests_rate":savings_intrests_rate,
    "savings_marketing_efficiency":savings_marketing_efficiency,
    "savings_requests":savings_requests})[:-1]
df_loans=pd.DataFrame({
    "global_expenditure":global_expenditure,
    "loans_expenditure":loans_expenditure,
    "loans_intrests_rate":loans_intrests_rate,
    "loans_marketing_efficiency":loans_marketing_efficiency,
    "loans_requests":loans_requests})[:-1]

In [289]:
df_savings

,global_expenditure,savings_expenditure,savings_intrests_rate,savings_marketing_efficiency,savings_requests
0,4500000,1200000,0.030,0.6915,902795.1
1,4500000,1400000,0.035,0.6576,951090.4
2,4500000,1200000,0.035,0.7680,993601.1
3,3300000,900000,0.032,0.9026,837795.4
4,3600000,1440000,0.030,0.9684,1114117.8


In [26]:
df_loans["global_expenditure"]

0    4500000
1    4500000
2    4500000
3    3300000
4    3600000
Name: global_expenditure, dtype: int64

In [290]:
y_savings=df_savings["savings_requests"]
X_savings=df_savings.drop(["savings_requests"],axis=1)

df_savings_pre=pd.DataFrame({
    "global_expenditure":[4500000],
    "savings_expenditure":[5000000],
    "savings_intrests_rate":[0.02],
    "savings_marketing_efficiency":[0.9854]
})

best_regr_savings=find_best_regr(X_savings,y_savings,df_savings_pre,2376077.1)

NameError: name 'find_best_regr' is not defined

In [ ]:
y_loans=df_loans["loans_requests"]
X_loans=df_loans.drop(["loans_requests"],axis=1)

df_loans_pre=pd.DataFrame({
    "global_expenditure":[4500000],
    "savings_expenditure":[1500000],
    "savings_intrests_rate":[0.147],
    "savings_marketing_efficiency":[0.9854]
})

best_regr_loans=find_best_regr(X_loans,y_loans,df_loans_pre,158472.6)

In [ ]:
best_regr_savings.coef_

In [ ]:
best_regr_loans.coef_